In [64]:
import os
import re
import json
with open("data_raw/vanocni_filmy_csfd.json", "r", encoding="utf-8") as filmy:
    filmy = json.loads(filmy.read())

In [66]:
len(filmy)

66

In [68]:
filmy

[['Adéla ještě nevečeřela',
  'https://www.csfd.cz/film/5981-adela-jeste-nevecerela/prehled/',
  'film'],
 ['Anděl Páně', 'https://www.csfd.cz/film/215414-andel-pane/prehled/', 'film'],
 ['Ať přiletí čáp, královno!',
  'https://www.csfd.cz/film/108859-at-prileti-cap-kralovno/prehled/',
  'tv'],
 ['Bohouš', 'https://www.csfd.cz/film/8186-bohous/prehled/', 'tv'],
 ['Byl jednou jeden král...',
  'https://www.csfd.cz/film/10089-byl-jednou-jeden-kral/prehled/',
  'film'],
 ['Cesta do Ameriky',
  'https://www.csfd.cz/film/5577-cesta-do-ameriky/prehled/',
  'film'],
 ['Co takhle svatba, princi?',
  'https://www.csfd.cz/film/108854-co-takhle-svatba-princi/prehled/',
  'tv'],
 ['Císař a tambor',
  'https://www.csfd.cz/film/5345-cisar-a-tambor/prehled/',
  'film'],
 ['Císařův pekař',
  'https://www.csfd.cz/film/3094-cisaruv-pekar-pekaruv-cisar/prehled/',
  'film'],
 ['Dalskabáty, hříšná ves aneb Zapomenutý čert',
  'https://www.csfd.cz/film/197358-dalskabaty-hrisna-ves-aneb-zapomenuty-cert/prehl

In [70]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import asyncio

async def load_page_with_playwright(url):
    """Load a web page using Playwright and parse with BeautifulSoup"""
    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        # Navigate to URL
        print(f"Loading: {url}")
        await page.goto(url, wait_until="domcontentloaded")
        
        # Get page content
        content = await page.content()
        
        # Close browser
        await browser.close()
        
        # Parse with BeautifulSoup
        soup = BeautifulSoup(content, 'html.parser')
        return soup

In [72]:
async def csfd(url):
    
    soup = await load_page_with_playwright(url)

    id = url.split("/")[-3]
    origin = [x.strip() for x in soup.find(class_="origin").text.split(",")]

    hraji, scenar, rezie = None, None, None
    
    for d in soup.find_all("div"):
        if d.find('h4'):
            if "Hrají:" in d.find('h4').text:
                hraji = [x.text.strip() for x in d.find_all("a")]
            if "Scénář:" in d.find('h4').text:
                scenar = [x.text.strip() for x in d.find_all("a")]
            if "Režie:" in d.find('h4').text:
                rezie = [x.text.strip() for x in d.find_all("a")]

    for a in soup.find_all("a"):
        if "Hodnocení" in a.text:
            if "(" in a.text:
                if ")" in a.text:
                    pocet_hodnoceni = int(''.join(re.findall(r'\d+', a.find(class_="counter").text)))
    
    film = {
        'id': id,
        'titul': soup.find("title").text.split("(")[0].strip(),
        'zanry': [x.strip().lower() for x in soup.find(class_="genres").text.split("/")],
        'zeme': origin[0],
        'rok': int(origin[1]),
        'delka': int(origin[2].split(' ')[0].strip()),
        'rezie': rezie,
        'scenar': scenar,
        'hraji': hraji,
        'anotace': soup.find(class_="plot-full").text.strip(),
        'hodnoceni': int(soup.find(class_="film-rating-average").text.strip().replace('%','')),
        'pocet_hodnoceni': pocet_hodnoceni
    }
    
    return film

In [74]:
os.makedirs("data_raw/csfd",exist_ok=True)
soubory = os.listdir("data_raw/csfd")

os.makedirs("data_raw/csfd",exist_ok=True)
for f in filmy:
    url = f[1]
    soubor = url.split("/")[-3] + ".json"
    if soubor not in soubory:
        print(soubor)
        try:
            url = f[1]
            metadata = await csfd(url)
            print(metadata)
            with open(os.path.join('data_raw/csfd',soubor), 'w+', encoding="utf-8") as exp:
                exp.write(json.dumps(metadata))
                print("Uloženo")
        except Exception as e:
            print(e)
        soubory = os.listdir("data_raw/csfd")

3083-anton-spelec-ostrostrelec.json
Loading: https://www.csfd.cz/film/3083-anton-spelec-ostrostrelec/prehled/
{'id': '3083-anton-spelec-ostrostrelec', 'titul': 'Anton Špelec, ostrostřelec', 'zanry': ['komedie'], 'zeme': 'Československo', 'rok': 1932, 'delka': 80, 'rezie': ['Martin Frič'], 'scenar': ['Josef Neuberg', 'Emil Artur Longen'], 'hraji': ['Vlasta Burian', 'Růžena Šlemrová', 'Jaroslav Marvan', 'Theodor Pištěk', 'Jindřich Plachta', 'Jiří Dréman', 'Ella Nollová', 'Karel Postranecký', 'František Kreuzmann st.', 'Alexander Třebovský', 'Čeněk Šlégl', 'Karel Schleichert', 'Viktor Nejedlý st.', 'Emanuel Hříbal', 'Ferdinand Jarkovský', 'Anči Pírková', 'Robert Ford', 'Václav Menger', 'Jan W. Speerger', 'Josef Kytka', 'Antonín Frič', 'Luigi Hofman', 'Josef Klapuch', 'Jindřich Edl', 'F. X. Mlejnek', 'Jan Richter', 'František Říha', 'Eduard Šlégl', 'Bohdan Lachman', 'Josef Sládek', 'Emil Dlesk', 'Antonín Volný', 'Marie Oliaková', 'Antonín Šolc', 'Josef Prček', 'více'], 'anotace': 'Nejzdaři

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


{'id': '9411-jak-se-budi-princezny', 'titul': 'Jak se budí princezny', 'zanry': ['pohádka'], 'zeme': 'Československo', 'rok': 1977, 'delka': 83, 'rezie': ['Václav Vorlíček'], 'scenar': ['Bohumila Zelenková'], 'hraji': ['Jiří Sovák', 'Milena Dvorská', 'Marie Horáková', 'Jan Hrušínský', 'Jan Kraus', 'Vladimír Menšík', 'František Filipovský', 'Stella Zázvorková', 'Oldřich Velen', 'Libuše Švormová', 'Václav Postránecký', 'Slávka Hozová', 'Marie Brožová', 'Jiří Lír', 'Vlastimil Hašek', 'Miloš Vavruška', 'Evelyna Steimarová', 'Jana Bittlová', 'Marie Crhová', 'Dana Kubálková', 'Vladimír Pospíšil', 'Oskar Hák', 'Zdeněk Dryšl', 'Jan Houdek', 'Václav Hrůza', 'Miloš Stránský', 'Alois Klejzar', 'Jan Valihrach', 'Ladislav Sýkora', 'Vlastimil Slezák', 'František Švihlík', 'Jan Kuželka', 'Jiří Růžička ml.', 'Pavla Maršálková', 'Lubomíra Willigová', 'Blanka Křížová', 'Stanislav Benda', 'Jiří Kraus', 'Jaromír Crha', 'František Miroslav Doubrava', 'Karel Fiala', 'Bohumil Koška', 'Josef Kubín', 'Vladimír